In [23]:
import numpy as np
from igraph import *
import pandas as pd
import time

In [2]:
data = pd.read_csv('../data/jimi/jimi_data/graph_data.csv')

/Applications/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,13,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.columns

Index([u'Unnamed: 0', u'orderId', u'userId', u'tag1', u'tag2', u'l_user_phone',
       u'workplace', u'createTime', u'goodsName', u'oriPri', u'num', u'price',
       u'firstPay', u'relMobile', u'province', u'city', u'region',
       u'order_ldeviceId', u'order_ip', u'order_mac', u'GPS', u'GPS_100',
       u'GPS_1000'],
      dtype='object')

## 1.使用正常的属性直接建图

### 1.1 使用UserId这个属性进行测试

In [94]:
attributes = ['userId','l_user_phone','order_ip','GPS','GPS_100','GPS_1000','order_ldeviceId'] #可以直接使用的attribute
# workplace,relMobile需要经过处理 有频率很大的数据，会生成很多无用边

In [95]:
# Initial Graph
g = Graph()
g.add_vertices(data.shape[0])

In [60]:
attribute = 'userId'
X = data[data[attribute].notnull()][attribute]
dic = {}
for i in X.index:
    try:
        dic[X.loc[i]].append(i)
    except:
        dic[X.loc[i]] = []
        dic[X.loc[i]].append(i)
#一个attribute的dic完成
for L in dic.values(): #dic.values()是list的集合
    if len(L)>1:#说明存在边
        for i in xrange(len(L)):
            for j in xrange(i+1,len(L)):
                g.add_edges([(L[i],L[j])])

In [96]:
print g.ecount()

0


### 1.2 全部属性建图

In [ ]:
for attribute in attributes:
    print attribute,
    X = data[data[attribute].notnull()][attribute]
    dic = {}
    for i in X.index:
        try:
            dic[X.loc[i]].append(i)
        except:
            dic[X.loc[i]] = []
            dic[X.loc[i]].append(i)
    #一个attribute的dic完成
    for L in dic.values(): #dic.values()是list的集合
        if len(L)>1:#说明存在边
            for i in xrange(len(L)):
                for j in xrange(i+1,len(L)):
                    g.add_edges([(L[i],L[j])])
    print 'graph edge #:',g.ecount()

userId graph edge #: 25135
l_user_phone graph edge #: 50270
order_ip graph edge #: 60023
GPS graph edge #: 78381
GPS_100 graph edge #: 167427
GPS_1000

In [92]:
print g.ecount()

96004


## 2.使用workplace,relMobile, order_mac 建图

In [90]:
# relMobile
X = data[data['relMobile'].notnull()]
X = X[X['relMobile']!='d41d8cd98f00b204e9800998ecf8427e']['relMobile']
dic = {}
for i in X.index:
    try:
        dic[X.loc[i]].append(i)
    except:
        dic[X.loc[i]] = []
        dic[X.loc[i]].append(i)

for L in dic.values(): #dic.values()是list的集合
    if len(L)>1:#说明存在边
        for i in xrange(len(L)):
            for j in xrange(i+1,len(L)):
                g.add_edges([(L[i],L[j])])
print g.ecount()

78410


In [91]:
X = data[data['workplace'].notnull()]
X1 = X[X['workplace'] != '个体']
X2 = X1[X1['workplace']!='自由职业']
X3 = X2[X2['workplace']!='个体户']
X = X3[X3['workplace']!='无']['workplace']
dic = {}
for i in X.index:
    try:
        dic[X.loc[i]].append(i)
    except:
        dic[X.loc[i]] = []
        dic[X.loc[i]].append(i)

for L in dic.values(): #dic.values()是list的集合
    if len(L)>1:#说明存在边
        for i in xrange(len(L)):
            for j in xrange(i+1,len(L)):
                g.add_edges([(L[i],L[j])])

In [88]:
#'order_mac'
X1 = data[data['order_mac'].notnull()]
X = X1[X1['order_mac']!='未知']['order_mac']
dic = {}
for i in X.index:
    try:
        dic[X.loc[i]].append(i)
    except:
        dic[X.loc[i]] = []
        dic[X.loc[i]].append(i)

for L in dic.values(): #dic.values()是list的集合
    if len(L)>1:#说明存在边
        for i in xrange(len(L)):
            for j in xrange(i+1,len(L)):
                g.add_edges([(L[i],L[j])])

##  3. 生成unipartite graph 得到Adjlist并去重

In [ ]:
AdjList = g.get_adjlist()
AdjList_new = []
for AdjNode in AdjList:
    AdjList_new.append(set(AdjNode))

In [ ]:
import cPickle as pickle
with open('Unipartite_Graph','w') as f:
    pickle.dump(g)
with open('')